### Main Patch SLA Extraction

### Configuration

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Basic Imports

In [2]:
import ee
import geemap
import logging

ee.Initialize()

logging.basicConfig(level=logging.INFO)

In [3]:
import sys
sys.path.append('../')

### Local Imports

In [4]:
from sla.prep import PreProcessor
from sla.cloud_shadow_mask import add_cloud_shadow
from sla.classifier import decision_tree
from sla.hill_shadow import add_hill_shadow
from sla.main_patches import extract_sla_patch

from sla.settings import ING, ALOS, SRTM

- Here we run our pipeline and generate the image collection till the classifier step.
- We need to pass a single image to our `extract_sla_patch` to see intermitted results.

- If we pass the entire image collections using `map` then Earth Engine will throw error!

In [5]:
ing_id = "G718255O411666S"
start_year = ee.Number(2022)
end_year = ee.Number(2022)
doy_start = ee.Number(50)
doy_end = ee.Number(200)
cloudiness = ee.Number(50)
coverage = 50
hsboolean = 0
hs_start = ee.Number(50)
hs_end = ee.Number(200)
dem = "SRTM"

preprocessor = PreProcessor(
    ing_id=ing_id,
    start_year=start_year,
    end_year=end_year,
    doy_start=doy_start,
    doy_end=doy_end,
    cloudiness=cloudiness,
    coverage=coverage,
    hsboolean=hsboolean,
    dem=dem,
)

preprocessed = preprocessor.execute()

print("processing cloud shadow mask...")
cloud_shadow_collection = preprocessed.map(add_cloud_shadow)

print("processing hill shadow...")
hill_shadow_collection = add_hill_shadow(
        image_collection=cloud_shadow_collection, hs_start=hs_start, hs_end=hs_end
    )

print("classifying with decision tree...")
classified_collection = hill_shadow_collection.map(decision_tree)

processing cloud shadow mask...
processing hill shadow...
classifying with decision tree...


Note:

- `classified_collection` is the image collection after the decision tree classifier step
- to debug main patch we take the first image and pass it to `extract_sla_patch`

In [7]:
sla = classified_collection.map(extract_sla_patch)

In [9]:
sla.getInfo()

{'type': 'FeatureCollection',
 'columns': {'Considered Area for MP-SLA Extraction': 'Number',
  'ID_Glacier': 'String',
  'Ratio Area v/o Snow or Ice': 'Number',
  'SLA MP-Approach': 'Object',
  'Snow Cover Ratio': 'Number',
  'date_MSxlsx': 'Number',
  'ice_area': 'Number',
  'otsu': 'Number',
  'stdDev SLA, 0 means no touch between Main patches [m]': 'Object',
  'system:time_start': 'Object',
  'total_area': 'Number'},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '2_2_20220311T142741_20220311T144201_T19GBQ',
   'properties': {'Considered Area for MP-SLA Extraction': 90.62658334740753,
    'ID_Glacier': 'G718255O411666S',
    'Ratio Area v/o Snow or Ice': 0.06683008212432395,
    'SLA MP-Approach': 1640,
    'Snow Cover Ratio': 0.8914034791420368,
    'date_MSxlsx': 44631.61418981482,
    'ice_area': 129422059.2635747,
    'otsu': 0.47,
    'stdDev SLA, 0 means no touch between Main patches [m]': 0,
    'system:time_start': 1647009866834,
    'total_area': 5921}},
 

In [ ]:
# Map = geemap.Map(center=[40,-100], zoom=10)
# Map.addLayer(sla)
# Map.centerObject(sla, zoom=10)
# Map

In [ ]:
image = classified_collection.first()

Our first image looks like this

You can run this notebook for any intermitted image you want. There are 26 in total

In [ ]:
image.getInfo()

In [ ]:
sla = extract_sla_patch(image)

So the intermediate variables have been written to an `exports` folder in the `notebooks` directory.
You can open it directly or inspect it like this...

In [ ]:
import json

with open('exports/elevation.json') as f:
    elevation = json.load(f)

In [ ]:
elevation

Final SLA looks like this...

In [ ]:
sla.getInfo()

In [ ]:
# Map = geemap.Map(center=[40,-100], zoom=10)
# Map.addLayer(sla)
# Map.centerObject(sla, zoom=10)
# Map

### The End